In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv("API_KEY")

### 2. Get the Review Corpus

In [3]:
import requests

def fetch_reviews(token , number_of_reviews = 5, page_num = 1):
    url = "https://data.g2.com/api/v1/survey-responses"
    headers = {
        "Authorization": f"Token token={token}",
        "Content-Type": "application/vnd.api+json"
    }

    
    params = {

        "page[size]": f"{number_of_reviews}",
        "page[number]": f"{page_num}"  
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        return data['data']
    else:
        print(f"Error: {response.status_code} - {response.text}")
        
        return None


response = fetch_reviews(token, 100, 1)

### 3. Preprocess the Review Corpus

In [4]:
from datetime import datetime

def change_to_dd_mm_yyyy(rfc3339_date):
    dt_object = datetime.fromisoformat(rfc3339_date)
    return dt_object.strftime('%d%m%Y')

def create_review_dictionary(reviews):
    review_dict = {}
    for review in reviews:
        attributes = review["attributes"]
        review_id = review["id"]
        try:
            review_dict[review_id] = {
                "ease_of_doing_business_with": attributes["secondary_answers"]["ease_of_doing_business_with"]["value"] if "ease_of_doing_business_with" in attributes["secondary_answers"] else None,
                "country_name": attributes["country_name"],
                "votes_up": attributes["votes_up"],
                "votes_down": attributes["votes_down"],
                "votes_total": attributes["votes_total"],
                "star_rating": attributes["star_rating"],
                "time": change_to_dd_mm_yyyy(attributes["submitted_at"]),
                "likes": attributes["comment_answers"]["love"]["value"],
                "dislikes": attributes["comment_answers"]["hate"]["value"]
            }
        except:
            pass
    return review_dict

review_dict = create_review_dictionary(response)

In [14]:
likes_dict = {}


for review_id, review in review_dict.items():
    likes = review["likes"]
    dislikes = review["dislikes"]
    if likes in likes_dict:
        likes_dict[review_id].append(likes)
    else:
        likes_dict[review_id] = [likes]
        
        


### 4. Name Imputation

In [15]:
import spacy

nlp = spacy.load("en_core_web_lg")

def change_name(sentence : str):
    doc = nlp(sentence)
    replaced_sentence = sentence
    for entity in doc.ents:
        if entity.label_ == "PERSON" :
            replaced_sentence = replaced_sentence.replace(entity.text, "person")
        if entity.label_ == "ORG" :
            replaced_sentence = replaced_sentence.replace(entity.text, "organization")            
           

    return replaced_sentence

In [16]:
for review_id in likes_dict:
    for like in likes_dict[review_id]:
        likes_dict[review_id] = change_name(like)
        

In [17]:
likes_dict

{'18532': "organization has helped our customers publicly validate us to prospects and has helped us build pipeline and be considered for opportunities where we likely would not have been found otherwise. Also it has aided as a great validation point for customers in meetings and demo's as to our market profile and position.  It has helped us need less reference calls and enabled our customers to have a more open and valid place to voice their opinion on us.",
 '67896': 'It showcases a wide range of vendors with unbiased and helpful reviews for evaluating solutions within your organization. This helps companies accelerate their marketing efforts and sales cycles when shared effectively. \r\n\r\nThe smart grid is also a really great way to show how your company is perceived in the market place.\r\n\r\nQuarterly reports are very detailed and are helpful overviews for companies trying to evaluate vendors. ',
 '177131': 'Great way to build our brand in a new space, by leveraging user succe

### 5. Feature Extraction

In [18]:
import os
import requests

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"

def get_headers(api_key):
    headers = {"Authorization": f"Bearer {api_key}"}
    return headers

def query(payload, api_key):
    headers = get_headers(api_key)
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def extract_features(review, api_key):
    preprompt = "Here is a review : "
    endprompt = "List the features what user liked, give keyword or 2 word phrases only. Give the output in format features: feature1, feature2, feature3, ..."
    
    response = query({
        "inputs": preprompt + review + endprompt,
        "temperature": 0.9,
    }, api_key)
    
    return response

# Example usage:
API_KEY = os.getenv("HF_KEY4")  # Assuming you retrieve the API key from environment variables
review = "This product is amazing! I love it."
features = extract_features(review, API_KEY)
print(features)


[{'generated_text': 'Here is a review : This product is amazing! I love it.List the features what user liked, give keyword or 2 word phrases only. Give the output in format features: feature1, feature2, feature3, ...Reusable, Mini, Lightweight, Powerful, Charges quickly, Compact, Portable, Durable, Rechargeable, Long Lasting, Versatile, Affordable.\n\nFeatures: Compact, Portable, Lightweight, Rechargeable, Long Lasting, Versatile, Affordable, Powerful.'}]


In [19]:
features_dict = {}

for review_id in likes_dict:
    features_dict[review_id] = extract_features(likes_dict[review_id], API_KEY)

In [ ]:
import json

# Your feature_dict should be populated as per your previous code

# Specify the file path where you want to save the JSON file
json_file_path = "feature_dict_lite.json"

# Write feature_dict to a JSON file
with open(json_file_path, "w") as json_file:
    json.dump(features_dict, json_file)

print("Feature dictionary lite saved as JSON successfully.")


Feature dictionary saved as JSON successfully.


In [ ]:
copy_feature_dict = feature_dict.copy()

In [ ]:
for review_id in copy_feature_dict:
    review = copy_feature_dict[review_id][0]["generated_text"].lower()
    if review.find("feature1, feature2, feature3, ...") != -1:
        copy_feature_dict[review_id]= review[review.find("feature1, feature2, feature3, ...")+len("feature1, feature2, feature3, ..."):].strip()
        
    
    

In [ ]:
for review_id in copy_feature_dict:
    review = copy_feature_dict[review_id]
    
    if review.find("features:") != -1:
        copy_feature_dict[review_id] = review[review.find("features:")+len("features:"):].strip()


In [ ]:
for review_id in copy_feature_dict:
    # remove punctuations and escape sequences from the string and remove digits
    review = copy_feature_dict[review_id]
    review = review.replace("\n", "")
    review = review.replace("\r", "")
    review = review.replace(".", "")
    review = review.replace("\"", "")
    review = review.replace("\'", "")
    review = review.replace("/", "")
    review = review.replace("(", "")
    review = review.replace(")", "")
    review = review.replace("*", "")
    review = review.replace("-", "")
    review = review.replace(":", "")
    
    for letter in review:
        if letter.isdigit():
            review = review.replace(letter, "")
    
    copy_feature_dict[review_id] = review
    
    
    

In [ ]:
for review_id in copy_feature_dict:
    review = copy_feature_dict[review_id].strip()
    review = review.split(",")
    
    sorted_review = sorted(review, key = len)
    
    
    for i in range(len(sorted_review)):
        feature = sorted_review[i].strip()
        
        if len(feature.split(" ")) > 3:
            feature = ""
        
        sorted_review[i] = feature
    
    copy_feature_dict[review_id] = sorted_review

In [ ]:
for review_id in copy_feature_dict:
    print(copy_feature_dict[review_id])

['market profile', 'market presence', 'public validation', 'pipeline generation', 'position validation', 'open customer voice', 'less reference calls', 'opportunity consideration']
['api', 'security', 'reporting', 'reporting', 'automation', 'support so', 'automation', 'flexibility', 'data accuracy', 'user interface', 'cost effective', 'user interface', 'customer service', '', 'scalabilityvendor b compliance', '', '', '']
['']
['optimization', 'endtoend service', 'review campaigns', 'goals achievement', 'process implementation']
['innovation', 'continuous flow', 'excellent service', 'committed clients', '']
['support', 'review requests', 'made themselves available', 'promotions proactive outreach', '']
['automation', 'less work', 'high volume', 'organization', 'high quality']
['support', 'initial setup', 'automated process', '', '']
['social sharing', 'easy referrals', 'positive reviews', 'high conversion rate', '']
['outreach', 'reviews', 'building', 'promoters', 'platforms', 'top revi

In [ ]:
for review_id in copy_feature_dict:
    features = copy_feature_dict[review_id]
    if features == [""]:
        copy_feature_dict[review_id] = []
        continue
    
    # Create a new list to store the non-empty features
    cleaned_features = []
    
    for feature in features:
        if feature != "":
            cleaned_features.append(feature)
    
    # Update the features in the dictionary
    copy_feature_dict[review_id] = cleaned_features


In [ ]:
corpus =[]
for review_id in copy_feature_dict:
    corpus.append(copy_feature_dict[review_id])

### preprocessing

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import  PorterStemmer, WordNetLemmatizer
import string

# Function for text preprocessing
def preprocess_text(phrase):
    # Tokenization
    res = []
    for i in phrase:
        
        tokens = word_tokenize(i)
        stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        # Convert to lowercase
        tokens = [word.lower() for word in tokens]
        
        # Remove punctuation
        table = str.maketrans('', '', string.punctuation)
        tokens = [word.translate(table) for word in tokens]
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        
        # Stem tokens
        #tokens = [stemmer.stem(token) for token in tokens]
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        tokens = [word for word in tokens if word not in stop_words ]
        tokens = [word for word in tokens if not word.isdigit()]

        
        tokens= " ".join(tokens)
                
        res.append(tokens)
    return res



### TFIDF VECTORIZATION + SIMILAR WORDS CLUSTER CREATION

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity


preprocessed_dataset = []
# Preprocess each document in the corpus
for i in range(len(corpus)):
    preprocessed_dataset.append(preprocess_text(corpus[i]))

# Flatten the preprocessed dataset
flattened_dataset = [item for sublist in preprocessed_dataset for item in sublist]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(flattened_dataset)

# Calculate pairwise cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix)

# K-means clustering
k = 45# Number of clusters (adjust as needed)
kmeans = KMeans(n_clusters=k)
kmeans.fit(cosine_sim)

# Extracting important features from each cluster
clusters = {}
for i, label in enumerate(kmeans.labels_):
    if label not in clusters:
        clusters[label] = [flattened_dataset[i]]
    else:
        clusters[label].append(flattened_dataset[i])

# Print out important features in each cluster
for cluster_id, features in clusters.items():
    print(f"Cluster {cluster_id}:")
    for feature in features:
        print(f"- {feature}")
    print()


Cluster 16:
- market profile
- market presence
- public validation
- pipeline generation
- position validation
- le reference call
- opportunity consideration
- api
- security
- flexibility
- cost effective
- scalabilityvendor b compliance
- optimization
- endtoend service
- process implementation
- innovation
- continuous flow
- committed client
- made available
- promotion proactive outreach
- le work
- high volume
- high quality
- initial setup
- automated process
- social sharing
- high conversion rate
- outreach
- building
- promoter
- engagement
- gamification
- costeffective
- dashboard view
- robust security
- automated alert
- aipowered sentiment analysis
- automate process
- explore net promoter
- move forward
- favorable response
- effective communication
- favorable response rate
- saas solution industry
- identifies best practice
- provided template
- engineered success
- saw
- list
- 
- 
- love
- software
- growing
- pushing
- finding
- product
- exactly
- enhance
- enabl

In [ ]:
from collections import Counter

# Function to nominate two features from a cluster
def nominate_features(cluster):
    # Flatten the list of features in the cluster
    all_features = [feature for features in cluster for feature in features.split()]
    
    # Count the frequency of each term
    term_frequency = Counter(all_features)
    
    # Get the most frequent and second most frequent terms
    most_frequent_terms = term_frequency.most_common(6)
    
    return most_frequent_terms

# Dictionary to store the nominated features for each cluster
nominated_features = {}

# Nominate features for each cluster
for cluster_id, cluster_features in clusters.items():
    most_frequent_terms = nominate_features(cluster_features)
    nominated_features[cluster_id] = most_frequent_terms

# Print nominated features for each cluster
for cluster_id, nominated in nominated_features.items():
    print(f"Cluster {cluster_id}:")
    print("Nominated Features:")
    for rank, (term, frequency) in enumerate(nominated, start=1):
        
        print(f"{rank}. {term} (Frequency: {frequency})")
    print()


Cluster 16:
Nominated Features:
1. feature (Frequency: 40)
2. product (Frequency: 39)
3. clear (Frequency: 31)
4. tool (Frequency: 30)
5. sale (Frequency: 28)
6. communication (Frequency: 27)

Cluster 21:
Nominated Features:
1. customer (Frequency: 72)
2. success (Frequency: 10)
3. segmentation (Frequency: 5)
4. experience (Frequency: 4)
5. insight (Frequency: 4)
6. advocacy (Frequency: 3)

Cluster 12:
Nominated Features:
1. reporting (Frequency: 48)
2. realtime (Frequency: 8)
3. detailed (Frequency: 5)
4. custom (Frequency: 4)
5. comprehensive (Frequency: 3)
6. robust (Frequency: 2)

Cluster 24:
Nominated Features:
1. automation (Frequency: 22)
2. tool (Frequency: 3)
3. email (Frequency: 2)
4. workflow (Frequency: 2)
5. business (Frequency: 1)
6. process (Frequency: 1)

Cluster 3:
Nominated Features:
1. support (Frequency: 33)
2. team (Frequency: 6)

Cluster 23:
Nominated Features:
1. data (Frequency: 84)
2. intent (Frequency: 28)
3. buyer (Frequency: 16)
4. analysis (Frequency: 10)
5

In [ ]:
def find_word_freq(cluster):
    all_features = [feature for features in cluster for feature in features.split()]
    term_frequency = Counter(all_features)
    return term_frequency


cluster_frequency = {}
for cluster_id, cluster_features in clusters.items():
    term_frequency = find_word_freq(cluster_features)
    cluster_frequency[cluster_id] = term_frequency


In [ ]:
# Update clusters dictionary to include feature importance
for cluster_id, cluster in clusters.items():
    clusters[cluster_id] = []
    for feature in cluster:
        features = feature.split()
        feature_importance = sum(cluster_frequency[cluster_id].get(word, 0) for word in features)
        clusters[cluster_id].append({"feature": feature, "importance": feature_importance})

In [ ]:
# Sort clusters based on the total importance
sorted_clusters = sorted(clusters.items(), key=lambda x: sum(feature["importance"] for feature in x[1]), reverse=True)

# Sort features within each cluster based on their importance
for cluster_id, cluster_features in sorted_clusters:
    sorted_features = sorted(cluster_features, key=lambda x: x["importance"], reverse=True)
    clusters[cluster_id] = sorted_features

# Print sorted clusters
for cluster_id, cluster_features in sorted_clusters:
    print(f"Cluster {cluster_id}:")
    for feature in cluster_features:
        print(f"- {feature['feature']} (Importance: {feature['importance']})")
    print()


Cluster 16:
- market profile (Importance: 49)
- market presence (Importance: 32)
- public validation (Importance: 11)
- pipeline generation (Importance: 5)
- position validation (Importance: 14)
- le reference call (Importance: 19)
- opportunity consideration (Importance: 8)
- api (Importance: 3)
- security (Importance: 14)
- flexibility (Importance: 10)
- cost effective (Importance: 20)
- scalabilityvendor b compliance (Importance: 3)
- optimization (Importance: 12)
- endtoend service (Importance: 15)
- process implementation (Importance: 34)
- innovation (Importance: 2)
- continuous flow (Importance: 5)
- committed client (Importance: 9)
- made available (Importance: 7)
- promotion proactive outreach (Importance: 22)
- le work (Importance: 5)
- high volume (Importance: 14)
- high quality (Importance: 18)
- initial setup (Importance: 8)
- automated process (Importance: 30)
- social sharing (Importance: 32)
- high conversion rate (Importance: 24)
- outreach (Importance: 7)
- building (

In [ ]:
# Remove duplicates within each cluster
for cluster_id, cluster_features in clusters.items():
    unique_features = []
    seen_features = set()
    for feature in cluster_features:
        if feature["feature"] not in seen_features:
            unique_features.append(feature)
            seen_features.add(feature["feature"])
    clusters[cluster_id] = unique_features

# Sort clusters based on the total importance
sorted_clusters = sorted(clusters.items(), key=lambda x: sum(feature["importance"] for feature in x[1]), reverse=True)

# Sort features within each cluster based on their importance
for cluster_id, cluster_features in sorted_clusters:
    sorted_features = sorted(cluster_features, key=lambda x: x["importance"], reverse=True)
    clusters[cluster_id] = sorted_features

# Print sorted clusters with duplicates removed
for cluster_id, cluster_features in sorted_clusters:
    print(f"Cluster {cluster_id}:")
    for feature in cluster_features:
        print(f"- {feature['feature']} (Importance: {feature['importance']})")
    print()


Cluster 16:
- new product feature (Importance: 98)
- effective communication tool (Importance: 75)
- clear product description (Importance: 74)
- product page credibility (Importance: 73)
- product comparison dashboard (Importance: 73)
- simple product page (Importance: 70)
- feature custom brand (Importance: 69)
- feature flexible customization (Importance: 68)
- various profile feature (Importance: 65)
- sale enablement tool (Importance: 64)
- detailed product ranking (Importance: 64)
- product comparison table (Importance: 63)
- feature affordable pricing (Importance: 63)
- product profile (Importance: 63)
- feature comparison (Importance: 62)
- advanced security feature (Importance: 61)
- product comparison (Importance: 61)
- new feature release (Importance: 60)
- product feedback (Importance: 60)
- product page (Importance: 59)
- clear communication (Importance: 58)
- detailed product description (Importance: 57)
- feature generator software (Importance: 57)
- brand safety feature

In [ ]:
# Pick top 2 features from each cluster
top_features_per_cluster = {}

for cluster_id, cluster_features in clusters.items():
    sorted_features = sorted(cluster_features, key=lambda x: x["importance"], reverse=True)
    top_features_per_cluster[cluster_id] = sorted_features[:2]

# Print top 2 features from each cluster
for cluster_id, top_features in top_features_per_cluster.items():
    print(f"Cluster {cluster_id}:")
    for feature in top_features:
        print(f"- {feature['feature']} (Importance: {feature['importance']})")
    print()


Cluster 16:
- new product feature (Importance: 98)
- effective communication tool (Importance: 75)

Cluster 21:
- customer success manager (Importance: 84)
- proactive customer success (Importance: 84)

Cluster 12:
- realtime reporting (Importance: 56)
- detailed reporting (Importance: 53)

Cluster 24:
- automation tool (Importance: 25)
- email automation (Importance: 24)

Cluster 3:
- support team (Importance: 39)
- team support (Importance: 39)

Cluster 23:
- buyer intent data (Importance: 128)
- valuable intent data (Importance: 114)

Cluster 20:
- user friendly interface (Importance: 72)
- easy user interface (Importance: 68)

Cluster 6:
- excellent customer service (Importance: 70)
- responsive customer service (Importance: 61)

Cluster 42:
- incentivized review campaign (Importance: 39)
- review campaign management (Importance: 39)

Cluster 1:
- business goal (Importance: 21)
- goal achievement (Importance: 14)

Cluster 11:
- verified user review (Importance: 129)
- user review (